In [1]:
MODEL_DIR = None
MASK_CLS = 'ilm.mask.hierarchical.MaskHierarchical'
# MASK_CLS = 'ilm.mask.hierarchical_dailydialog.MaskHierarchical'

In [3]:
# Download pretrained model

if MODEL_DIR is None:
#     !python acl20_repro.py model sto ilm | bash
#     MODEL_DIR = '/tmp/ilm/models/sto_ilm'
    MODEL_DIR = 'train'
#     MODEL_DIR = 'train-naive'

In [48]:
from collections import Counter
import os
import random
import importlib
import pickle
import sys
import re
from tqdm.notebook import tqdm

from ilm.datasets import Dataset, get_dataset
import ilm.mask
from ilm.mask.util import mask_cls_str_to_type
from ilm.mask.util import apply_masked_spans
from ilm.mask.util import masked_spans_bounds_valid, masked_spans_overlap
from create_ilm_examples import randomly_mask_dataset, randomly_mask_document

# import importlib
# importlib.reload(create_ilm_examples)
# Set seed
seed = 0
if seed is None:
    seed = random.randint(0, 1e6)
    print('Random seed {}'.format(seed))
    random.seed(seed)

mask_type = mask_cls_str_to_type(MASK_CLS)
print(mask_type)
masker = mask_type(0.25, mask_paragraph_p=0.0, mask_document_p=0.0)


<class 'ilm.mask.hierarchical.MaskHierarchical'>


In [5]:
# Prepare tokenizer

import os
import pickle

import ilm.tokenize_util

tokenizer = ilm.tokenize_util.Tokenizer.GPT2
with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
    additional_ids_to_tokens = pickle.load(f)
additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
print(ilm.tokenize_util.encode('no , i dont <|infill_word|><|infill_word|> .<|infill_sentence|>', tokenizer))
try:
    lenvocab = ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
except ValueError:
    print('Already updated')
print(ilm.tokenize_util.encode('no , i dont <|infill_word|><|infill_word|> .<|infill_sentence|>', tokenizer))
doc_tokens = (ilm.tokenize_util.tokenize('no , i dont <|infill_word|><|infill_word|> .<|infill_sentence|>', tokenizer))
doc_tokens_ids = ilm.tokenize_util.tokens_to_ids(doc_tokens, tokenizer=tokenizer)
print(doc_tokens_ids)

print(additional_tokens_to_ids, lenvocab)

tokenizer_state = ilm.tokenize_util._get_tokenizer_state(tokenizer)
# for k,v in additional_tokens_to_ids.items():
#     tokenizer.add_special_tokens([k])
# tokenizer_state


[3919, 837, 1312, 17666, 1279, 91, 10745, 359, 62, 4775, 91, 6927, 91, 10745, 359, 62, 4775, 91, 29, 764, 27, 91, 10745, 359, 62, 34086, 594, 91, 29]
[3919, 837, 1312, 17666, 1279, 91, 10745, 359, 62, 4775, 91, 6927, 91, 10745, 359, 62, 4775, 91, 29, 764, 27, 91, 10745, 359, 62, 34086, 594, 91, 29]
[3919, 837, 1312, 17666, 1279, 91, 10745, 359, 62, 4775, 91, 6927, 91, 10745, 359, 62, 4775, 91, 29, 764, 27, 91, 10745, 359, 62, 34086, 594, 91, 29]
{'<|startofinfill|>': 50257, '<|endofinfill|>': 50258, '<|speaker1|>': 50259, '<|speaker2|>': 50260, '<|context|>': 50261, '<|endofcontext|>': 50262, '<|response|>': 50263, '<|infill_document|>': 50264, '<|infill_paragraph|>': 50265, '<|infill_sentence|>': 50266, '<|infill_ngram|>': 50267, '<|infill_word|>': 50268} 50269


In [6]:
# Load model

import torch
from transformers import GPT2LMHeadModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)
model.eval()
_ = model.to(device)

from lm_scorer.models.auto import AutoLMScorer as LMScorer

# # Available models
# list(LMScorer.supported_model_names())
# # => ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl", distilgpt2"]

# # Load model to cpu or cuda
device = "cuda:0" if torch.cuda.is_available() else "cpu"
batch_size = 1
scorer = LMScorer.from_pretrained("gpt2", device=device, batch_size=batch_size)


In [7]:
device

'cuda:0'

In [8]:
# Create context

context = """
Math Class
Chris was bad at _. _ _ _ He ended up passing the test.
""".strip()

context = """<|context|> <speaker1> while the curtains are being made , i can start having people look at the kitchen . i can't stand that old kitchen . i won't\
be able to cook there . i don't want to use that electric stove . <speaker2> we need to find an interior decorating company to redecorate the kitchen . i believe in portland there are shops that specialize in kitchen renovation . i will look in the yellow pages .\
i'd like a kitchen mostly in ivory and light green . <speaker1> i agree . the colors must be soft and pleasant . you should feel comfortable when you cook our dinners . <speaker2>\
me ? cook our dinners ? hah ! you will be cooking , dear . you will cook . <|end of context|> <|response|> <|speaker2|> no , i don't  _  _ in my job . _""".strip()



context_ids = ilm.tokenize_util.encode(context, tokenizer)

# Replace blanks with appropriate tokens from left to right
_blank_id = ilm.tokenize_util.encode(' _', tokenizer)[0]
# context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_word|>']
# context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
# context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
# context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']

context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_word|>']
context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_word|>']
context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']


print(ilm.tokenize_util.decode(context_ids, tokenizer))
# print(context_ids)

print(ilm.tokenize_util.encode('no , i dont <|infill_word|> <|infill_word|> . <|infill_sentence|>', tokenizer))

<|context|> <speaker1> while the curtains are being made , i can start having people look at the kitchen . i can't stand that old kitchen . i won'tbe able to cook there . i don't want to use that electric stove . <speaker2> we need to find an interior decorating company to redecorate the kitchen . i believe in portland there are shops that specialize in kitchen renovation . i will look in the yellow pages .i'd like a kitchen mostly in ivory and light green . <speaker1> i agree . the colors must be soft and pleasant . you should feel comfortable when you cook our dinners . <speaker2>me ? cook our dinners ? hah ! you will be cooking , dear . you will cook . <|end of context|> <|response|> <|speaker2|> no , i don't <|infill_word|> <|infill_word|> in my job .<|infill_sentence|>
[3919, 837, 1312, 17666, 1279, 91, 10745, 359, 62, 4775, 91, 29, 1279, 91, 10745, 359, 62, 4775, 91, 29, 764, 1279, 91, 10745, 359, 62, 34086, 594, 91, 29]


In [22]:
import importlib
importlib.reload(ilm.infer)

from ilm.infer import infill_with_ilm, infill_naive_with_ilm

infill_function = infill_with_ilm
# infill_function = infill_naive_with_ilm


test_s = (ilm.tokenize_util.decode(context_ids, tokenizer))
print(test_s)
etest_s =ilm.tokenize_util.encode(test_s, tokenizer)
# print(etest_s)
test_ss = (ilm.tokenize_util.decode(etest_s, tokenizer))
# print(context_ids)
# print(etest_s)
# print(test_s)
# print(test_ss)
print('begin\n')
generated = infill_function(
    model,
    additional_tokens_to_ids,
    context_ids,
    num_infills=4)
for g in generated:
    print('-' * 80)
    print(ilm.tokenize_util.decode(g, tokenizer))
    


<|context|> <speaker1> while the curtains are being made , i can start having people look at the kitchen . i can't stand that old kitchen . i won'tbe able to cook there . i don't want to use that electric stove . <speaker2> we need to find an interior decorating company to redecorate the kitchen . i believe in portland there are shops that specialize in kitchen renovation . i will look in the yellow pages .i'd like a kitchen mostly in ivory and light green . <speaker1> i agree . the colors must be soft and pleasant . you should feel comfortable when you cook our dinners . <speaker2>me ? cook our dinners ? hah ! you will be cooking , dear . you will cook . <|end of context|> <|response|> <|speaker2|> no , i don't <|infill_word|> <|infill_word|> in my job .<|infill_sentence|>
begin

--------------------------------------------------------------------------------
<|context|> <speaker1> while the curtains are being made , i can start having people look at the kitchen . i can't stand that o

In [54]:
import importlib
import sys
importlib.reload(sys.modules['create_ilm_examples'])
from create_ilm_examples import randomly_mask_dataset, randomly_mask_document


docs_masked = []

error_to_count_total = Counter()
num_examples_per_document=3
max_num_retries=5

# docs = [' we need to find an interior decorating company to redecorate the kitchen . i believe in portland there are shops that specialize in kitchen renovation .']
docs_in = ['Encourage your child in her interests and hobbies even if they are things that you know little about.']
# docs_in = ['Encourage know little about.']

num_retries_total = 0

def get_masked_spans(docs):
    result = []
    docs_masked=[]
#     for doc in tqdm(docs):
    for doc in (docs):
        doc_masks, error_to_count = randomly_mask_document(
            doc,
            masker,
            num_examples_per_document,
            max_num_retries=max_num_retries,
              min_masked_spans=None,
              max_masked_spans=None,
              random_sample_down_to_max=True,
              ensure_valid_bounds_in_spans=True,
              ensure_nonoverlapping_spans=True,
            )
#         print(doc_masks, error_to_count)
        docs_masked.append((doc, doc_masks))
        for k, v in error_to_count.items():
            error_to_count_total[k] += v
#     print(len(docs_masked[0]))
    i = 0
    for doc, examples in docs_masked:
#         if len(examples) == 0:
#             continue

        for masked_spans in examples:
#         masked_spans = random.choice(examples)
            mask_span_type_to_str = {t:'<|{}|>'.format(str(t)) for t, _, _ in masked_spans}
            context, answers = apply_masked_spans(
                doc,
                masked_spans,
                mask_span_type_to_str)
            context_copy = context
            if context_copy in ['<|MaskHierarchicalType.DOCUMENT|>', '<|MaskHierarchicalType.PARAGRAPH|>', '<|MaskHierarchicalType.SENTENCE|>']:
                if random.random() >0.5: context_copy = doc + ' ' + context
                else: context_copy = context + ' ' + doc
            result.append([doc,context_copy])
#             for _ in range(4):
#               print('-' * 80)
#             print(' ' * 36 + '-' * 8)
#             print(' ' * 36 + 'ORIGINAL')
#             print(' ' * 36 + '-' * 8)
#             print(doc)
#             print(' ' * 36 + '-' * 7)
#             print(' ' * 36 + 'CONTEXT')
#             print(' ' * 36 + '-' * 7)
#             print(context)
#             print(' ' * 36 + '-' * 7)
#             print(' ' * 36 + 'ANSWERS')
#             print(' ' * 36 + '-' * 7)
#             for i, (span_type, span) in enumerate(answers):
#               print(mask_span_type_to_str[span_type])
#               print(span)
#               if i != len(answers) - 1:
#                 print('-' * 20)

        return result

outs = get_masked_spans(docs_in)
print(outs)

[['Encourage your child in her interests and hobbies even if they are things that you know little about.', 'Encourage your child in her interests and <|MaskHierarchicalType.WORD|> even <|MaskHierarchicalType.WORD|> they are things <|MaskHierarchicalType.WORD|> you know little about.'], ['Encourage your child in her interests and hobbies even if they are things that you know little about.', 'Encourage your child in <|MaskHierarchicalType.NGRAM|> and hobbies even if they <|MaskHierarchicalType.NGRAM|> <|MaskHierarchicalType.NGRAM|>.'], ['Encourage your child in her interests and hobbies even if they are things that you know little about.', 'Encourage your child in her interests and hobbies even if they are things that you know little about. <|MaskHierarchicalType.SENTENCE|>']]


In [43]:


context = """<|context|> <speaker1> while the curtains are there , i can start having people look at the kitchen . i can't stand that old kitchen . i won't\
be able to cook there . i don't want to use that electric stove . <speaker2> we need to find an interior decorating company to redecorate the kitchen . i believe in portland there are shops that specialize in kitchen renovation . i will look in the yellow pages .\
i'd like a kitchen mostly in ivory and light green . <speaker1> i agree . the colors must be soft and pleasant . you should feel comfortable when you cook our dinners . <speaker2>\
 me ? cook our dinners ? hah ! you will be cooking , dear . you will cook . <|end of context|> <|response|> <|speaker2|> """.strip()

stop_words = ['another', 'once', 'thereupon', 'whom', 'regarding', 'first', 'anyhow', 'whence', 'else', 'might', 'themselves', '’s', 'of', 'side', "'m", 'top', 'ours', 'will', 'whole', 'sixty', 'if', 'but', 'serious', 'cannot', 'became', 'about', 'do', 'take', 'an', 'being', 'throughout', 'after', 're', 'were', 'must', 'thence', 'whether', 'whereafter', 'hundred', 'again', 'still', 'further', 'above', 'third', 'them', 'any', 'why', "'ll", 'wherein', 'towards', 'every', 'five', 'most', 'into', 'meanwhile', 'may', 'onto', 'neither', 'namely', 'fifteen', 'i', 'below', 'they', 'without', 'him', 'never', 'give', 'forty', 'own', 'thus', 'whereby', 'yourself', 'itself', 'somewhere', 'via', 'full', 'next', 'been', 'always', 'put', 'whereupon', 'because', 'so', 'under', 'during', 'than', 'several', 'upon', 'very', '’d', 'something', "n't", 'ten', '‘m', 'though', 'anything', 'fifty', 'all', 'seemed', 'well', 'twenty', 'more', 'amongst', 'wherever', 'name', 'am', 'therein', 'much', 'among', 'less', 'when', 'except', 'hereafter', 'has', 'along', 'seems', 'now', 'up', 'sometimes', 'alone', 'ca', 'everything', 'enough', 'himself', 'everyone', '‘ve', 'quite', '‘re', 'elsewhere', 'whoever', 'it', 'back', 'me', 'otherwise', 'perhaps', 'latter', 'on', 'already', 'across', 'whither', 'what', 'within', '‘d', 'n’t', "'ve", 'that', 'nevertheless', 'someone', 'nowhere', 'empty', 'out', 'some', 'really', 'off', 'each', 'mostly', 'hence', 'yet', 'are', 'using', 'nothing', 'yourselves', 'no', "'re", 'besides', 'over', '‘ll', 'sometime', 'becomes', 'before', 'anywhere', 'by', 'seem', 'for', 'us', 'where', 'many', 'these', 'he', 'toward', 'her', 'should', 'doing', 'ever', 'nor', 'three', 'between', 'can', 'same', 'whereas', 'until', 'either', 'their', 'due', '’ll', 'beside', 'few', 'the', 'was', 'which', 'its', 'just', 'our', 'your', 'say', 'noone', 'front', 'against', 'down', 'such', 'anyway', 'also', 'everywhere', 'two', 'together', 'and', 'others', 'bottom', 'eight', 'we', 'my', "'d", 'whatever', 'six', 'indeed', 'did', 'other', 'becoming', 'afterwards', 'from', 'thereafter', 'too', 'you', 'behind', 'mine', 'a', 'thereby', 'not', 'to', 'nobody', 'be', 'done', 'then', 'at', 'even', '’re', 'here', 'various', 'make', 'twelve', 'how', 'as', 'since', 'there', 'call', 'somehow', 'she', 'in', 'anyone', 'almost', 'moreover', 'beyond', 'herself', 'yours', 'hereby', 'both', 'nine', 'latterly', 'herein', 'have', 'eleven', 'while', 'his', 'please', 'n‘t', 'move', 'those', 'get', 'could', 'beforehand', 'this', 'is', 'per', 'although', 'hers', 'made', '’m', 'often', 'ourselves', 'therefore', 'whose', 'keep', 'only', 'none', 'seeming', 'one', '’ve', 'hereupon', 'whenever', 'unless', '‘s', 'does', 'had', 'would', 'however', 'formerly', 'see', 'used', 'show', 'around', 'part', "'s", 'become', 'least', 'thru', 'last', 'who', 'rather', 'myself', 'through', 'former', 'four', 'or', 'with', 'go']

def get_infill_substitutes(sentence):
#     '<|infill_document|>': 50259, '<|infill_paragraph|>': 50260, '<|infill_sentence|>': 50261, '<|infill_ngram|>': 50262, '<|infill_word|>
    sentence = sentence.replace('<|MaskHierarchicalType.PARAGRAPH|>', ' <|infill_sentence|> ')
    sentence = sentence.replace('<|MaskHierarchicalType.DOCUMENT|>', ' <|infill_sentence|> ')
    sentence = sentence.replace('<|MaskHierarchicalType.SENTENCE|>', ' <|infill_sentence|> ')
    sentence = sentence.replace('<|MaskHierarchicalType.NGRAM|>', ' <|infill_ngram|> ')
#     sentence = sentence.replace('<|MaskHierarchicalType.WORD|>', '<|infill_word|>')
    #word replacement with ngram to make it longer
    sentence = sentence.replace('<|MaskHierarchicalType.WORD|>', ' <|infill_ngram|> ')
    sentence = sentence.replace('   ', ' ')
    sentence = sentence.replace('  ', ' ')
    return sentence

_blank_id = ilm.tokenize_util.encode(' _', tokenizer)[0]

def get_context_ids(data_input):
    
    types_of_blanks = ['<|infill_document|>', '<|infill_paragraph|>', '<|infill_sentence|>', '<|infill_ngram|>', '<|infill_word|>']
    
    list_of_blanks = []
    s = data_input
    while True:
        m = re.search('\<\|infill_(.+?)\|\>',s)
        if m:
            found = m.group(1)
            list_of_blanks.append('<|infill_'+found+'|>')
            s = s.replace('<|infill_'+found+'|>', ' _',1)
        else:
            break
    data_input = s
    
    context_ids = ilm.tokenize_util.encode(data_input, tokenizer)
    for type_blank in list_of_blanks:
        context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids[type_blank]

    return context_ids

def test_pure_infill(infill_sent):
    infills = re.findall('\<\|infill_(.+?)\|\>',infill_sent)
    words = infill_sent.split()
#     print(len(infills), len(words))
    if len(words)<=len(infills):
        return True
    
    if len(infills)<=1:
        if len(infills)==1 and infills[0] in ['<|infill_ngram|>', '<|infill_word|>']:
            return True
        if len(infills)==0: return True
    
    
    return False

def get_infilled_responses(context, sentence_infill_pairs, num_infills=1, verbose=False):
    list_generated_responses = []
    for o in sentence_infill_pairs:
        infill_sent = get_infill_substitutes(o[1])
        is_high_infill = test_pure_infill(infill_sent)
        if is_high_infill:
            continue
#         print(o[1])
        if verbose: print(infill_sent)
        data_input = context + ' ' + infill_sent
        context_ids = get_context_ids(data_input)
#         print(context_ids)
#         print(data_input)
#         print(ilm.tokenize_util.decode(context_ids, tokenizer))
    #     print('-----')

        generated = infill_with_ilm(
            model,
            additional_tokens_to_ids,
            context_ids,
            num_infills=num_infills)
        for g in generated:
            generated_sent = ilm.tokenize_util.decode(g, tokenizer)
            generated_response = generated_sent.split('<|response|> <|speaker2|> ')[-1]
#             print('-' * 80)
#             print(generated_sent)
            if verbose: print(generated_response)
            list_generated_responses.append(generated_response)
        if verbose: print()
#     random.shuffle(list_generated_responses)
    
    return list_generated_responses



def get_infilled_responses_naive(context, sentence_infill_pairs, num_infills=2):
    list_generated_responses = []
    for o in sentence_infill_pairs:
        infill_sent = get_infill_substitutes(o[1])
#         print(infill_sent)
        is_high_infill = test_pure_infill(infill_sent)
        if is_high_infill:
            continue
#         print(o[1])
#         print(infill_sent)
        data_input = context + ' ' + infill_sent
        context_ids = get_context_ids(data_input)
#         print(data_input)
        
        generated = infill_naive_with_ilm(
            model,
            additional_tokens_to_ids,
            context_ids,
            num_infills=num_infills)
        for g in generated:
            generated_sent = ilm.tokenize_util.decode(g, tokenizer)
            generated_response = generated_sent.split('<|startofinfill|>')[-1]
#             print('-' * 80)
#             print(generated_sent)
#             print(generated_response)
            if generated_response=='': continue
            list_generated_responses.append(generated_response)
#         print()
#     random.shuffle(list_generated_responses)
    
    return list_generated_responses

# [[r,o[1]] for r,o in zip(get_infilled_responses(context,outs), outs)]
# responses_filled = get_infilled_responses_naive(context,outs)
responses_filled = get_infilled_responses(context, outs, verbose=True)

print(len(responses_filled))
print([o[1] for o in outs])
print(responses_filled)
# [[r,o[1]] for r,o in zip(responses_filled, outs)]

 <|infill_ngram|> in her interests and hobbies even if they <|infill_ngram|> know little <|infill_ngram|> .
 ok , you sound in her interests and hobbies even if they  aren't your know little  about .

Encourage your child in her interests <|infill_ngram|> <|infill_ngram|> little about.
Encourage your child in her interests  .  so long as she is told to be good little about.

 <|infill_sentence|> Encourage your child in her interests and hobbies even if they are things that you know little about.
 when i cook , i will put my clothes on . Encourage your child in her interests and hobbies even if they are things that you know little about.

Encourage <|infill_ngram|> <|infill_ngram|> interests and hobbies <|infill_ngram|> know little about.
Encourage  everyone to be responsible for their own diet and house cleaning .  the kitchen promotes healthy interests and hobbies  are full of healthy recipes . know little about.

 <|infill_ngram|> in <|infill_ngram|> even if they <|infill_ngram|> thi

In [44]:
import json
import re
import pprint
import logging
import requests
import math
import re
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

#https://www.scribendi.ai/comparing-bert-and-gpt-2-as-language-models-to-score-the-grammatical-correctness-of-a-sentence/
#https://github.com/simonepri/lm-scorer
p = re.compile(r'((?<=[\.\?!]\s)(\w+)|(^\w+))')
def cap(match):
    return(match.group().capitalize())

def read_json_data(filename):
    data = []
    with open(filename) as f:
        for line in f:
            data.append(json.loads(line))
            
    return data

def get_context_string(doc, history_len = 4):
    context_string = '<|context|>'
    sents = doc['context'][-history_len:]
    for i,sent in enumerate(sents):
        context_string += ' <|speaker' + str((i-(len(sents)+1)%2)%2 +1) + '|> ' + sent
    context_string += ' <|endofcontext|> <|response|> <|speaker2|> '
#     doc = doc['response'][:]
    
    return context_string


test_data = read_json_data('/home/ubuntu/Code/negaug/dataset/test.json')
train_data = read_json_data('/home/ubuntu/Code/negaug/dataset/train.json')
dev_data = read_json_data('/home/ubuntu/Code/negaug/dataset/dev.json')

for data in dev_data:
    data['context_string'] = get_context_string(data)
for data in test_data:
    data['context_string'] = get_context_string(data)
for data in train_data:
    data['context_string'] = get_context_string(data)

In [56]:
from tqdm.autonotebook import tqdm
test_data[0]

def check_ispoor_text(sentence):
    sentence_words = sentence.split()
    
    for i, word in enumerate(sentence_words):
        if i>1 and word==sentence_words[i-1]:
            return True
        
    return False

def get_lm_ppl(sentences):
    sentences = [p.sub(cap, r) for r in sentences]
    scores = scorer.sentence_score(sentences, log=True)
    scores_charlength = [s / (len(r)+2) for s,r in zip(scores, sentences)]
    scores_length = [s / (len(r.split())+2) for s,r in zip(scores, sentences)]

    return list(map(lambda x, y, z:(x,y,z), sentences, scores_charlength, scores_length)) 


def filter_clean_adv_set(adv_gen_neg_responses_t1, positive_responses, desired_number_responses = 5):
    
    #cleanup responses
    adv_gen_neg_responses = []
    for i, response in enumerate(adv_gen_neg_responses_t1):
        clean_response = response.strip()
        clean_response = clean_response.replace(' ,', ',')
        clean_response = clean_response.replace(' .', '.')
        clean_response = clean_response.replace(' ?', '?')
        clean_response = clean_response.replace('  ', ' ')
        clean_response = clean_response.replace(' !', '!')
        clean_response = clean_response.replace(' \' ', '\'')
        clean_response = clean_response.replace(' \'', '\'')
        clean_response= re.sub(' +',' ',clean_response)
        clean_response = re.sub('\$ ', '$', re.sub(' \%', '%', clean_response))
        clean_response = re.sub('\. (\d+)', r'.\1', clean_response)
        clean_response = p.sub(cap, clean_response)
#         clean_response = re.sub(r'($)?\s+(\d)(.)?\s+(\d)', r'\1\2', clean_response)
        adv_gen_neg_responses.append(clean_response)
    
    adv_gen_neg_responses_t1 = adv_gen_neg_responses
    
    positive_responses_wordset_list = []
    for pr in positive_responses:
        positive_responses_wordset_list.append(set(pr.split()))
    
    #get lm scores 
    lm_scores = get_lm_ppl(adv_gen_neg_responses_t1)
    
    good_responses = []
    responses_words_list = []
    threshold = 0.1
    sent_length_threshold = 7
    tries = 0
    while True:
        tries+=1
#         print('looped')
        for i, response in enumerate(adv_gen_neg_responses_t1):
            words_set = set(response.split())
            max_sim_score = -0.1
            #if sentence too small, decrease its chances
            if len(words_set)<=sent_length_threshold:
                max_sim_score = 0.89
                
            for old_response_words_set in responses_words_list:
                intersection_words = old_response_words_set.intersection(words_set)
                score = len(intersection_words)/len(words_set)
                max_sim_score = max(score, max_sim_score)

            #should not have too much overlap with positive responses
            for pos_response_words_set in positive_responses_wordset_list:
                interesection_words = pos_response_words_set.intersection(words_set)
                score = len(interesection_words)/len(words_set)  
#                 print(pos_response_words_set, words_set, score)
                if score >=0.8:
                    max_sim_score =0.89
            
            ##remove poor text, todo: add language model score
            #if check_ispoor_text(response):
            lm_sent = lm_scores[i][1]
            lm_sent_wordlevel = lm_scores[i][2]
#             print(lm_scores[i])
            if lm_sent<-1.4 or lm_sent_wordlevel<-5.0:
                max_sim_score = 0.89
                     
                
            if max_sim_score==-0.1 or max_sim_score<threshold:
#                 print('--addingresponse ', response, max_sim_score)
                good_responses.append(response)
                responses_words_list.append(words_set)
                
        threshold+=0.1
        if sent_length_threshold>2:
            sent_length_threshold-=1
        
        if len(good_responses)>=desired_number_responses:
            return good_responses[:desired_number_responses]
        
        if tries>4 or threshold>0.99:
            print('choosing randomly')
            return random.sample(adv_gen_neg_responses_t1, desired_number_responses)
        
#         print(good_responses)
    
    return good_responses[:desired_number_responses]

def get_maskable_candidates(positive_responses, context):
    candidates = []
    reserve = []
    for pr in positive_responses:
        prtokens = pr.lower().split()
        prtokens = [t for t in prtokens if t.replace('.', '') not in stop_words]
#         print(prtokens)
        if len(prtokens)<2:
            reserve.append(pr)
        else:
            candidates.append(pr)
    
    for pr in context:
        prtokens = pr.lower().split()
        prtokens = [t for t in prtokens if t not in stop_words]
        if len(prtokens)<2:
            reserve.append(pr)
        else:
            candidates.append(pr)
    
#     print(candidates, reserve)
    return candidates + reserve


def add_modgt_responses(test_data_point, train_data, verbose=False):
    positive_responses = test_data_point['positive_responses']
    context = test_data_point['context']
    context_string = test_data_point['context_string']
    #print(positive_responses)
    masked_spans = []
    candidates = get_maskable_candidates(context, positive_responses)
    candidates = candidates[:len(positive_responses)]
    for pr in candidates:
        masked_spans += get_masked_spans([pr])
    random_train = random.choice(train_data)
    random_context = random_train['context']
    random_context_string = random_train['context_string']
    if verbose: print('random_context_string ',random_context_string)
#     print('now infill')
    adv_gen_neg_responses_t1 = get_infilled_responses(random_context_string, masked_spans, num_infills=3, verbose=verbose)
#     adv_gen_neg_responses_t1 = get_infilled_responses_naive(random_context_string, masked_spans, num_infills=2)

#     print(positive_responses)
    if verbose: print(adv_gen_neg_responses_t1)
#     print('now filter')
    adv_gen_neg_responses_t1 = filter_clean_adv_set(adv_gen_neg_responses_t1, positive_responses)
    test_data_point['adv_gen_neg_responses_t1'] = adv_gen_neg_responses_t1 
    
    
    
    
# for i,data_point in enumerate(test_data):
#     if i%100==0: print(i)
#     if i==10: break
#     print(i)
# #     positive_responses = test_data['positive_responses']
#     add_modgt_responses(data_point, train_data)
# #     pprint.pprint(data_point, indent=1)
# #     break

# dest_file = "/hdd/Code/Others/gitrepos/Dialogue-Evaluation-with-BERT/dataset/test_advt1.json"
# output_file = open(dest_file, 'w', encoding='utf-8')
# for dic in train_data:
#     json.dump(dic, output_file) 
#     output_file.write("\n")


def save_data_to_disk(data, dest_file):
    output_file = open(dest_file, 'w', encoding='utf-8')
    for i,dic in enumerate(data):
        if i==10:break
        json.dump(dic, output_file) 
        output_file.write("\n")

data = test_data
dest_file = "/home/ubuntu/Code/negaug/dataset/test_pp.json"


for i,data_point in tqdm(enumerate(data), total=len(data)):
#     if i%100==0:
#         print(i)
    if i==10: break
#     positive_responses = test_data['positive_responses']
    add_modgt_responses(data_point, data)
#     pprint.pprint(data_point, indent=1)

save_data_to_disk(data, dest_file)


now infill
now filter
now infill
now filter
now infill
now filter
now infill
now filter
now infill
now filter
now infill
now filter
now infill
now filter
now infill
now filter
now infill
now filter
now infill
now filter


In [1]:
import spacy
sp = spacy.load("en")

all_stopwords = sp.Defaults.stop_words
all_stopwords

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.